### This is a second notebook due to environment issues

In [20]:
import numpy as np
import lab3_tools as tools
import lab3_proto as proto
import lab1_proto as proto1
import lab1_tools as tools1
import lab2_proto as proto2
import lab2_tools as tools2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.utils import np_utils
from keras.utils import np_utils

%reload_ext autoreload
%autoreload 2

In [21]:
stateList = np.load("stateList.npy", allow_pickle=True).tolist()

# 4.6 continuation

## Data conversion

In [22]:
output_dim = len(stateList)
#----
trainFile = np.load("flatDataTrain.npz")
lmfcc_train_x=trainFile["lmfcc_train_x"].astype('float32')
mspec_train_x=trainFile["mspec_train_x"].astype('float32')
dlmfcc_train_x=trainFile["dlmfcc_train_x"].astype('float32')
dmspec_train_x=trainFile["dmspec_train_x"].astype('float32')
train_y=np_utils.to_categorical(trainFile["train_y"],output_dim)
#print(lmfcc_train_x.shape,mspec_train_x.shape,dlmfcc_train_x.shape,dmspec_train_x.shape,train_y.shape) #| Shapes checked and OK
#----
valFile = np.load("flatDataVal.npz")
lmfcc_val_x=valFile["lmfcc_val_x"].astype('float32')
mspec_val_x=valFile["mspec_val_x"].astype('float32')
dlmfcc_val_x=valFile["dlmfcc_val_x"].astype('float32')
dmspec_val_x=valFile["dmspec_val_x"].astype('float32')
val_y=np_utils.to_categorical(valFile["val_y"],output_dim)
#print(lmfcc_val_x.shape,mspec_val_x.shape,dlmfcc_val_x.shape,dmspec_val_x.shape,val_y.shape)
#----
testFile = np.load("flatDataTest.npz")
lmfcc_test_x=testFile["lmfcc_test_x"].astype('float32')
mspec_test_x=testFile["mspec_test_x"].astype('float32')
dlmfcc_test_x=testFile["dlmfcc_test_x"].astype('float32')
dmspec_test_x=testFile["dmspec_test_x"].astype('float32')
test_y=np_utils.to_categorical(testFile["test_y"],output_dim)
#print(lmfcc_test_x.shape,mspec_test_x.shape,dlmfcc_test_x.shape,dmspec_test_x.shape,test_y.shape)

# 5. Phoneme Recognition with Deep Neural Networks

In [23]:
#features = lmfcc_train_x
#labels = train_y

features_val_lmfcc = lmfcc_val_x
features_val_mspec = mspec_val_x
features_val_dlmfcc = dlmfcc_val_x
features_val_dmspec = dmspec_val_x
labels_val = val_y

#feature_dim = features.shape[1]
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Dense(80,activation=tf.nn.relu,input_shape=(feature_dim,)))
#model.add(tf.keras.layers.Dense(70,activation=tf.nn.relu)) #choose ReLu since it's faster to compute compared to Sigmoid
#model.add(tf.keras.layers.Dense(output_dim,activation=tf.nn.softmax)) #softmax distributes probabilitites across our states

#model.compile(loss="categorical_crossentropy",
            #optimizer="sgd",
            #metrics=["accuracy"])

#model.summary()


In [24]:
# = 2 #Goes through data set EPOCHS number of times 
#BATCH_SIZE = 256 #Each training step model will see BATCH_SIZE number of examples to guide and adjust parameters

#model.fit(features,labels,epochs=EPOCHS,batch_size=BATCH_SIZE)

In [25]:
EPOCHS = 2
BATCH_SIZE = 256
print("On lmfcc model")
model_lmfcc = proto.trainDNN(lmfcc_train_x,train_y,EPOCHS,BATCH_SIZE)
print("On mspec model")
model_mspec = proto.trainDNN(mspec_train_x,train_y,EPOCHS,BATCH_SIZE)
print("On dlmfcc model")
model_dlmfcc = proto.trainDNN(dlmfcc_train_x,train_y,EPOCHS,BATCH_SIZE)
print("On dmspec model")
model_dmspec = proto.trainDNN(dmspec_train_x,train_y,EPOCHS,BATCH_SIZE)

On lmfcc model
Epoch 1/2
5264/5264 [==============================] - 7s 1ms/step - loss: 2.3865 - accuracy: 0.3705
Epoch 2/2
5264/5264 [==============================] - 6s 1ms/step - loss: 1.7504 - accuracy: 0.4795
On mspec model
Epoch 1/2
5264/5264 [==============================] - 7s 1ms/step - loss: 2.4226 - accuracy: 0.3428
Epoch 2/2
5264/5264 [==============================] - 6s 1ms/step - loss: 1.9247 - accuracy: 0.4319
On dlmfcc model
Epoch 1/2
5264/5264 [==============================] - 7s 1ms/step - loss: 1.9605 - accuracy: 0.4528
Epoch 2/2
5264/5264 [==============================] - 6s 1ms/step - loss: 1.3708 - accuracy: 0.5741
On dmspec model
Epoch 1/2
5264/5264 [==============================] - 7s 1ms/step - loss: 1.9238 - accuracy: 0.4503
Epoch 2/2
5264/5264 [==============================] - 7s 1ms/step - loss: 1.3834 - accuracy: 0.5731


### Purpose of validation data? 

- Training data: Used to created weights of neural network
- Validation data: Used to compare different neural networks and choose the one that gives best result
- Test data: Simply used to test performance of the trained and chosen neural network. If not good results, start over. 

If you use the training and validation data as the same, the choice of neural network can be biased. Which means that the neural network was chosen as better just for that data. Real world data mighjt not have that connection to the neural network and might therefore not perform well. 

In [26]:
val_loss_lmfcc, val_acc_lmfcc = model_lmfcc.evaluate(features_val_lmfcc,labels_val)
print("val_acc_lmfcc:",val_acc_lmfcc) #Can increase with more epochs and more sofisticated model
print("val_loss_lmfcc:",val_loss_lmfcc)
val_loss_mspec, val_acc_mspec = model_mspec.evaluate(features_val_mspec,labels_val)
print("val_acc_mspec:",val_acc_mspec) #Can increase with more epochs and more sofisticated model
print("val_loss_mspec:",val_loss_mspec)
val_loss_dlmfcc, val_acc_dlmfcc = model_dlmfcc.evaluate(features_val_dlmfcc,labels_val)
print("val_acc_dlmfcc:",val_acc_dlmfcc) #Can increase with more epochs and more sofisticated model
print("val_loss_dlmfcc:",val_loss_dlmfcc)
val_loss_dmspec, val_acc_dmspec = model_dmspec.evaluate(features_val_dmspec,labels_val)
print("val_acc_dmspec:",val_acc_dmspec) #Can increase with more epochs and more sofisticated model
print("val_loss_dmspec:",val_loss_dmspec)

4996/4996 [==============================] - 3s 640us/step - loss: 1.6859 - accuracy: 0.4921
val_acc_lmfcc: 0.4921266436576843
val_loss_lmfcc: 1.6858845949172974
4996/4996 [==============================] - 3s 631us/step - loss: 1.8335 - accuracy: 0.4561
val_acc_mspec: 0.4560600221157074
val_loss_mspec: 1.8335018157958984
4996/4996 [==============================] - 3s 639us/step - loss: 1.2688 - accuracy: 0.6002
val_acc_dlmfcc: 0.6001576781272888
val_loss_dlmfcc: 1.268845796585083
4996/4996 [==============================] - 3s 643us/step - loss: 1.2760 - accuracy: 0.6012
val_acc_dmspec: 0.6012462377548218
val_loss_dmspec: 1.2759679555892944


## 5.2 Possible questions

- what is the influence of feature kind and size of input context window?
    - The input kind is important since that reveals the amount of "hidden data". Ex. the lmfcc has been stripped of some information compared to the mspecs. This can later be interpred by the hidden layers. 
    -   input context window $\color{red}{!!!}$
- what is the purpose of normalising (standardising) the input feature vectors depending on
the activation functions in the network?
    - Because otherwise some values that shouldn't activate the activation function will do so and vice versa. When normalising all values, one ensures that all data is represented in relation to each other.
    - Looking at what happens when one only normalises for each utterance, some parts of an utterance might trigger the activation function even though it wouldn't have if one normalised all utterances in relation to each other. This is why shorter utterances might be over represented when one only normalises for each utternace in isolation.
- what is the influence of the number of units per layer and the number of layers?
    - answer
- what is the influence of the activation function (when you try other activation functions
than ReLU, you do not need to reach convergence in case you do not have enough time)
    - answer
- what is the influence of the learning rate/learning rate strategy?
    - answer
- how stable are the posteriograms from the network in time?
    - answer
- how do the errors distribute depending on phonetic class?
    - answer